In [ ]:
%cd
%cd ../content
%rm -r *
!git clone https://github.com/alexanderskulikov/circuit_improvement.git
%cd circuit_improvement

In [ ]:
!pip install pycosat
!apt install libgraphviz-dev
!pip install pygraphviz
import pygraphviz
import sys
from timeit import default_timer as timer
from IPython.display import Image
import circuit_improvement
from circuit_improvement import *
from circuit import Circuit
from circuit_search import find_circuit, CircuitFinder
from main import *
from itertools import product
from math import ceil, log2

In [ ]:
#@title Improve circuit

improved_circuit = run_improve_circuit(add_sum5_suboptimal, 5, subcircuit_size=5, connected=True)
print(improved_circuit)
improved_circuit.draw('sum5')
Image('circuits/.images/sum5.png')


In [ ]:
#@title Find circuit by truth table

def sum_n(x):
    assert all(a in (0, 1) for a in x)
    s, t = sum(x), ceil(log2(len(x) + 1))
    return [(s >> i) & 1 for i in range(t)]

circuit_finder = CircuitFinder(dimension=3, number_of_gates=5, function=sum_n)
circuit = circuit_finder.solve_cnf_formula()
print(circuit)

In [ ]:
#@title Checking the circuit for correctness

circuit = Circuit(input_labels=['x1', 'x2', 'x3', 'x4', 'x5'], gates={})
x1, x2, x3, x4, x5 = circuit.input_labels
a0, a1 = add_sum3(circuit, [x1, x2, x3])
b0, b1 = add_sum3(circuit, [a0, x4, x5])
w1, w2 = add_sum2(circuit, [a1, b1])
circuit.outputs = [b0, w1, w2]
check_sum_circuit(circuit)
print(circuit)
circuit.draw('sum5')
Image('circuits/.images/sum5.png')